In [5]:
import os
import pandas as pd
from clickhouse_driver import Client

ck_host = os.environ.get('CK_HOST') or '127.0.0.1'

client = Client(host=ck_host, port=9000, user='wuhao', password='123456')



In [6]:
sql = """select sum(if(behavior_type = 'pv', 1, 0)) as pv, uniqExact(user_id) as uv
from test.user_behavior"""
df=client.query_dataframe(sql)
df

,pv,uv
0,89660671,987991


In [7]:
sql="""select DATE(timestamp) as day,
       sum(if(behavior_type = 'pv', 1, 0)) as pv,
       uniqExact(user_id) as uv
from test.user_behavior
group by DATE(timestamp)
order by day"""
df=client.query_dataframe(sql)
df

,day,pv,uv
0,2017-11-25,9353416,706641
1,2017-11-26,9567422,715516
2,2017-11-27,9041186,710094
3,2017-11-28,8842932,709257
4,2017-11-29,9210820,718922
5,2017-11-30,9358998,730597
6,2017-12-01,9718956,740139
7,2017-12-02,12329641,970401
8,2017-12-03,12237300,966977


###  每个用户的购物情况，加工到 user_behavior_count

In [8]:
sql="""drop table if exists test.user_behavior_count"""
client.execute(sql)
sql="""create view test.user_behavior_count as
(select user_id,
       sum(if(behavior_type='pv',1,0)) as pv,
       sum(if(behavior_type='fav',1,0)) as fav,
       sum(if(behavior_type='cart',1,0)) as cart,
       sum(if(behavior_type='buy',1,0)) as buy
       from test.user_behavior
group by user_id)"""
client.execute(sql)

[]

### 复购率：产生两次或两次以上购买的用户占购买用户的比例

In [9]:
sql="""select round(sum(if(buy > 1, 1, 0)) *100 / sum(if(buy > 0, 1, 0)),2)  as p
from test.user_behavior_count"""
df=client.query_dataframe(sql)
df

,p
0,66.01


### 用户行为转换率

In [10]:
sql="""select a.pv,
       a.fav,
       a.cart,
       a.fav + a.cart                     `fav+cart`,
       a.buy,
       round((a.fav + a.cart) / a.pv, 4)  pv2favcart,
       round(a.buy / (a.fav + a.cart), 4) favcart2buy,
       round(a.buy / a.pv, 4)             pv2buy
from (select sum(pv)   pv,
             sum(fav)  fav,
             sum(cart) cart,
             sum(buy)  buy
      from test.user_behavior_count) a"""
client.query_dataframe(sql)

,pv,fav,cart,fav_cart,buy,pv2favcart,favcart2buy,pv2buy
0,89660671,2888258,5530446,8418704,2015807,0.0939,0.2394,0.0225


### 用户行为习惯

In [11]:
sql="""select HOUR(timestamp)                  hour,
       sumIf(1, behavior_type = 'pv')   pv,
       sumIf(1, behavior_type = 'fav')  fav,
       sumIf(1, behavior_type = 'cart') cart,
       sumIf(1, behavior_type = 'buy') buy
from test.user_behavior
group by HOUR(timestamp)
order by hour"""
client.query_dataframe(sql)

,hour,pv,fav,cart,buy
0,0,3056935,105681,181807,57775
1,1,1421673,51374,85412,23169
2,2,769269,27791,46155,12012
3,3,524992,18384,32552,8025
4,4,449628,14793,28537,6748
5,5,581694,18992,37264,8135
6,6,1226653,40524,81305,18014
7,7,2228959,74666,147046,37679
8,8,3042342,103721,192036,64916
9,9,3728498,127976,229890,96134


### 一周用户的活跃分布

In [12]:
sql="""select  toDayOfWeek(timestamp) weekday,
       sumIf(1,behavior_type='pv') pv,
       sumIf(1,behavior_type='fav') fav,
       sumIf(1,behavior_type='cart') cart,
       sumIf(1,behavior_type='buy') buy
from test.user_behavior
where DATE(timestamp) between '2017-11-27' and '2017-12-03'
group by weekday
order by weekday"""
client.query_dataframe(sql)

,weekday,pv,fav,cart,buy
0,1,9041186,291221,541904,226834
1,2,8842932,289100,534157,211997
2,3,9210820,298587,551593,223068
3,4,9358998,302264,565015,221459
4,5,9718956,307115,623346,210010
5,6,12329641,396749,793569,257903
6,7,12237300,392197,774905,257754


### 最近一次购买

In [13]:
sql = """set allow_experimental_window_functions=1"""
client.execute(sql)
sql="""select user_id,
       dateDiff('day',max(timestamp),DATE('2017-12-04')) R,
       dense_rank() over( order by dateDiff('day',max(timestamp),DATE('2017-12-04'))) R_rank
from test.user_behavior
where behavior_type = 'buy'
group by user_id
limit 100;"""
client.query_dataframe(sql)

,user_id,R,R_rank
0,1006922,1,1
1,968242,1,1
2,749664,1,1
3,422576,1,1
4,728776,1,1
...,...,...,...
95,699008,1,1
96,276333,1,1
97,565092,1,1
98,130516,1,1


### 消费频率

In [14]:
sql = """set allow_experimental_window_functions=1;"""
client.execute(sql)
sql="""select user_id,
       count() as F,
       dense_rank() over(order by count() desc) as F_rank
from test.user_behavior
where behavior_type = 'buy'
group by user_id
limit 10;"""
client.query_dataframe(sql)

,user_id,F,F_rank
0,486458,262,1
1,866670,175,2
2,702034,159,3
3,107013,131,4
4,1014116,118,5
5,432739,112,6
6,500355,110,7
7,537150,109,8
8,1003412,100,9
9,919666,97,10


In [15]:
sql = """set allow_experimental_window_functions=1;"""
client.execute(sql)
sql="""select user_id,
       dateDiff('day', max(timestamp), DATE('2017-12-04')) R,
       dense_rank()                                        over( order by dateDiff('day',max(timestamp),DATE('2017-12-04'))) R_rank,
       count() as                                          F,
       dense_rank()                                        over(order by count() desc) as F_rank

from test.user_behavior
where behavior_type = 'buy'
group by user_id"""
client.query_dataframe(sql)

,user_id,R,R_rank,F,F_rank
0,80729,1,1,1,90
1,898002,1,1,3,88
2,781376,1,1,6,85
3,486458,1,1,262,1
4,367656,1,1,1,90
...,...,...,...,...,...
672399,656042,9,9,1,90
672400,657405,9,9,1,90
672401,920705,9,9,1,90
672402,438770,9,9,1,90
